In [139]:
import pandas as pd
# import sweetviz as sv
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
#from pandas_profiling import ProfileReport

In [140]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# 1. Description
The data we have is from the nurse study and contains labled data (stress/no stress). The data contains data from differente sensors: ACC (accelerometer), BVP (blood volume pulse), EDA (electrodermal activity), HR (hear rate), TEMP (temperature). In the following, we look at the features created with the FLIRT (https://flirt.readthedocs.io/en/latest/) library (see other script).

Note: there might be other potential features to be calculated on the raw data, for example via tsfresh (https://tsfresh.readthedocs.io/en/latest/index.html) or TSFEL (https://tsfel.readthedocs.io/en/latest/). However, FLIRT was specifically developed with the wrist sensor used in the two dataset used here, so we can reasonably expect it to produce meaningful features based on the available data.

# 2. Data Source

In [146]:
# load data - features calculated with Flirt with
# window_length = 60 and
# window_step_size = 10
df = pd.read_parquet('data-input/flirt-nurse-acc-bvp-eda-hr-temp-60-10.parquet')

In [175]:
df.shape

(26654, 266)

In [147]:
df.head(3)

,bvp_BVP_mean,bvp_BVP_std,bvp_BVP_min,bvp_BVP_max,bvp_BVP_ptp,bvp_BVP_sum,bvp_BVP_energy,bvp_BVP_skewness,bvp_BVP_kurtosis,bvp_BVP_peaks,...,hr_l2_n_sign_changes,hr_l2_iqr,hr_l2_iqr_5_95,hr_l2_pct_5,hr_l2_pct_95,hr_l2_entropy,hr_l2_perm_entropy,hr_l2_svd_entropy,subject,label
datetime,,,,,,,,,,,,,,,,,,,,,
2020-07-08 08:31:00,0.376078,207.186980,-1336.28,1333.44,2669.72,1444.14,1.648381e+08,-0.210934,8.208982,174,...,0.0,16.1900,20.1850,78.669,98.8540,4.090539,0.537376,0.040885,15,1
2020-07-08 08:31:10,0.813401,205.085789,-1336.28,1470.89,2807.17,3123.46,1.615136e+08,0.453604,10.256058,172,...,0.0,11.9525,18.4160,78.669,97.0850,4.091445,0.879136,0.054576,15,1
2020-07-08 08:31:20,-1.458391,183.187930,-823.18,1470.89,2294.07,-5600.22,1.288702e+08,1.219684,12.753530,178,...,0.0,6.7350,14.3385,78.669,93.0075,4.092899,0.998001,0.068318,15,1


In [148]:
# there are no missing values in the dataset
df.isnull().sum().value_counts()

0    266
dtype: int64

In [149]:
df['subject'].value_counts()

83    4289
E4    3516
DF    2610
7A    2566
CE    1918
BG    1901
EG    1716
6B    1612
6D    1508
5C    1432
F5    1196
94     830
7E     690
8B     586
15     284
Name: subject, dtype: int64

In [150]:
df['label'].value_counts(normalize=True)

1    0.839274
0    0.160726
Name: label, dtype: float64

# 3. Train-test split
We perform the train-test split before we conduct EDA on the train set. Thus, we avoid data leakage from the test set.

In [151]:
X = df.drop(columns=['subject', 'label'])
y = df[['label']]

In [152]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [153]:
df_train= pd.concat([X_train, y_train], axis=1)

# 4. EDA

## 4.1 Looking into data

In [154]:
# we do not have categorical features, only int (count) and float
df_train.dtypes.value_counts()

float64    224
int32       30
int64       11
dtype: int64

In [156]:
df_train.describe()

c:\Tools\anaconda3\envs\base3\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\Tools\anaconda3\envs\base3\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\Tools\anaconda3\envs\base3\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\Tools\anaconda3\envs\base3\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,bvp_BVP_mean,bvp_BVP_std,bvp_BVP_min,bvp_BVP_max,bvp_BVP_ptp,bvp_BVP_sum,bvp_BVP_energy,bvp_BVP_skewness,bvp_BVP_kurtosis,bvp_BVP_peaks,...,hr_l2_n_below_mean,hr_l2_n_sign_changes,hr_l2_iqr,hr_l2_iqr_5_95,hr_l2_pct_5,hr_l2_pct_95,hr_l2_entropy,hr_l2_perm_entropy,hr_l2_svd_entropy,label
count,21323.000000,21323.000000,21323.000000,21323.000000,21323.000000,21323.000000,2.132300e+04,21323.000000,21323.000000,21323.000000,...,21323.000000,21323.0,21323.000000,21323.000000,21323.000000,21323.000000,21323.000000,21323.000000,21323.000000,21323.000000
mean,-0.000777,81.896289,-480.656962,441.288654,921.945616,-2.289911,3.888219e+07,-0.275215,9.714851,147.879942,...,29.690475,0.0,5.034220,8.901721,79.926363,88.828083,4.069301,0.693065,0.028639,0.838859
std,0.689915,60.261046,370.484900,347.764399,695.564727,2201.859181,5.544580e+07,0.982297,12.333167,38.255196,...,6.448868,0.0,4.774026,7.483727,12.146316,14.762834,0.164423,0.330956,0.014352,0.367669
min,-19.124290,2.391237,-3096.160000,7.680000,16.710000,-21276.770000,4.267821e+03,-10.078009,-1.214924,11.000000,...,3.000000,0.0,0.060000,0.100000,51.636500,53.937000,1.945893,-0.000000,0.001518,0.000000
25%,-0.154538,35.961048,-667.850000,192.485000,406.155000,-581.305000,4.857504e+06,-0.645942,3.209126,123.000000,...,26.000000,0.0,1.657500,3.539000,72.980000,79.324000,4.092998,0.537376,0.018573,1.000000
50%,-0.000911,63.184157,-381.270000,353.580000,749.170000,-3.490000,1.508862e+07,-0.239709,6.138227,146.000000,...,30.000000,0.0,3.490000,6.676000,78.522000,86.470000,4.093934,0.831474,0.026046,1.000000
75%,0.156691,114.524212,-205.970000,588.040000,1265.590000,590.590000,4.943861e+07,0.152924,11.628347,170.000000,...,34.000000,0.0,6.850000,11.999500,85.797500,95.822500,4.094230,0.962146,0.035681,1.000000
max,20.174836,468.793463,-8.270000,2506.460000,5525.440000,22740.860000,5.208890e+08,10.436767,172.777141,479.000000,...,51.000000,0.0,50.912500,80.008500,165.049000,179.871500,4.094344,1.000000,0.265936,1.000000


In [157]:
# remove rows with only one value for each row
#overall_length = len(df_train)
columns = df_train.columns.tolist()
constant_columns = []

for c in columns:
    unique_in_column = len(df_train[c].unique())
    
    #if unique_in_column/overall_length < 0.1 and c != 'label' and c != 'subject':
    if unique_in_column == 1:
        constant_columns.append(c)

In [158]:
constant_columns

['bvp_BVP_entropy',
 'acc_l2_n_sign_changes',
 'temp_TEMP_n_sign_changes',
 'temp_l2_n_sign_changes',
 'hr_HR_n_sign_changes',
 'hr_l2_n_sign_changes']

In [159]:
# remove rows where we cannot calculate sandard deviation of the column

df_desc = df_train.describe()
columns_describe = df_train.describe().columns.tolist()
no_std_columns = []

for c in columns:
    std = df_desc[c]['std']

    #if unique_in_column/overall_length < 0.1 and c != 'label' and c != 'subject':
    if np.isnan(std):
        no_std_columns.append(c)

c:\Tools\anaconda3\envs\base3\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\Tools\anaconda3\envs\base3\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\Tools\anaconda3\envs\base3\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\Tools\anaconda3\envs\base3\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\Tools\anaconda3\envs\base3\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\Tools\anaconda3\envs\base3\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\Tools\anaconda3\env

In [160]:
no_std_columns

['bvp_BVP_entropy', 'acc_x_entropy', 'acc_y_entropy', 'acc_z_entropy']

In [170]:
columns_to_drop = list((set(constant_columns).union(set(no_std_columns))))

In [171]:
columns_to_drop

['acc_y_entropy',
 'bvp_BVP_entropy',
 'acc_z_entropy',
 'hr_HR_n_sign_changes',
 'hr_l2_n_sign_changes',
 'temp_l2_n_sign_changes',
 'acc_l2_n_sign_changes',
 'acc_x_entropy',
 'temp_TEMP_n_sign_changes']

In [172]:
df_train = df_train.drop(columns=columns_to_drop)

## 4.2 Correlations

In [ ]:
plt.figure(figsize=(35, 35))
corr = df_train.corr(method='spearman')
heatmap = sns.heatmap(corr.sort_values(by='label', ascending=False),
                      vmin=-1, vmax=1, annot=True, fmt='.1g', cmap='BrBG')
heatmap.set_title('Features correlating with stress label', fontdict={'fontsize':15}, pad=16);

The image is really large, so we load a screenshot here
![alt text](eda-nurse-acc-bvp-eda-hr-temp.jpg)

# 5. Documenting data lineage

The dataset contains
* BVP (blood volume pulse) sensor data in 64hz
* ACC: accelerometer data (x, y, z values) in 32hz
* EDA (electrodermal activity) in 4hz
* HR (hear rate) in 1hz
* TEMP (temperature) in 4hz

The dataset is labeled (stress/no stress).

Script ```01-extract-data-from-nurse-dataset``` downloads the nurse study dataset. It unzips all included files. From the raw data from the Empatica E4 wrist sensor, we use the devicely library (https://hpi-dhc.github.io/devicely/) for extracing the needed data and setting it in the correct timezone given in the publication about the data (https://www.nature.com/articles/s41597-022-01361-y). We look up the ground-truth labels and use the stress/no stress labels to label the sensor data. The results are stored as a parquet file.

In script ```02-calculate-features```, we calculate features with the FLIRT library (https://flirt.readthedocs.io/en/latest/). In this notebook you're currently reading, we perform EDA. In the following steps, we might want to go back to feature calculation and calculate other/more features.

# 6. Observations from EDA

### Looking into the dataset
* Given the windows size and step size, we have 26654 rows and 264 features.
* We could also calculate feature via tsfresh and/or TSFEL; and we could try different parameters for window_length and window_step_size when using FLIRT.
* There are no missing values.
* We have 84% of positive cases (stress) and 16% of negative cases in our data - we have to account for this when building and evaluating the model, e.g., by using appropriate evaluation metrics for imbalanced data.
* We do not have categorical variables, only numerical (count and float).

* There are a some columns that we drop, because they either have have the same value for each row, or it is impossible to calculate the standard deviation on them:
  * acc_y_entropy
  * bvp_BVP_entropy
  * acc_z_entropy
  * hr_HR_n_sign_changes
  * hr_l2_n_sign_changes
  * temp_l2_n_sign_changes
  * acc_l2_n_sign_changes
  * acc_x_entropy
  * temp_TEMP_n_sign_changes
* The ranges of the values are quite far from each other - we should normalize/standardize.

### Correlations
* There are several correlated features. Because of the amount of features, we should apply an automated method for deciding which features to keep.